In [ ]:
import pulp
import pandas as pd
from branch_and_bound import branch_and_bound

In [ ]:
prob = pulp.LpProblem("Test_plan", pulp.LpMinimize)

In [ ]:
import csv
df = pd.read_csv("food.csv")
with open('dish.csv') as f:
    reader = csv.reader(f)
    dish_list = list(reader)

In [ ]:
nutrient_limits = {'Energy': 2000,
          'Fat': 70,
          'Saturates': 20,
          'Carbohydrates': 260,
          'Sugars': 90,
          'Protein': 50
         }

In [ ]:
# temporary limits
category_limits = {'Cereal': (10, 10),
          'Tuber': (10, 10),
          'Fruit': (10, 10),
          'Vegetable': (10, 10),
          'Meat': (10, 10),
          'Fish': (10, 10),
          'Egg': (10, 10),
          'Legume': (10, 10),
          'Dairy': (10, 10),
          'Fat': (10, 10),
         }

In [ ]:
class dish:
    def __init__(self, name):
        self.name = name
        self.cost = 0
        self.nutrients = {nut:0 for nut in nutrient_limits.keys()}
        self.category = {cat: 0 for cat in category_limits.keys()}

In [ ]:
labels = list(df['Food'])
costs = dict(zip(labels,df['Cost']))
category = dict(zip(labels,df['Category']))

In [ ]:
nutrients = {}
for i in range(len(labels)):
    nutrients[labels[i]] = {}
    for nut in nutrient_limits.keys():
        nutrients[labels[i]][nut] = df[nut][i]*10

In [ ]:
dish_labels = []
dishes = []
for i in range(len(dish_list)):
    d = dish_list[i]
    dishes.append(dish(d[0]))
    dish_labels.append(d[0])
    for j in range(1,len(d),2):
        ingredient = d[j]
        quantity = float(d[j+1])/1000
        dishes[i].cost += costs[ingredient]*quantity
        for nut in dishes[i].nutrients.keys():
            dishes[i].nutrients[nut] += nutrients[ingredient][nut]*quantity
        for cat in dishes[i].category.keys():
            dishes[i].category[cat] += (category[ingredient]==cat)*quantity        

In [ ]:
foods_mon = pulp.LpVariable.dicts("Food_Mon",dish_labels,0)
#foods_tue = pulp.LpVariable.dicts("Food_Tue",dish_labels,0)
#foods_wed = pulp.LpVariable.dicts("Food_Wed",dish_labels,0)
#foods_thu = pulp.LpVariable.dicts("Food_Thu",dish_labels,0)
#foods_fri = pulp.LpVariable.dicts("Food_Fri",dish_labels,0)
#foods_sat = pulp.LpVariable.dicts("Food_Sat",dish_labels,0)
#foods_sun = pulp.LpVariable.dicts("Food_Sun",dish_labels,0)
#day_list = [foods_mon,foods_tue,foods_wed,foods_thu,foods_fri,foods_sat,foods_sun]
day_list = [foods_mon]

In [ ]:
prob += pulp.lpSum([sum([dish.cost*day[dish.name] for day in day_list]) for dish in dishes])


In [ ]:
for nut in nutrient_limits.keys():
    for day in day_list:
        prob += pulp.lpSum([dish.nutrients[nut] * day[dish.name] for dish in dishes]) >= nutrient_limits[nut]*0.5
        prob += pulp.lpSum([dish.nutrients[nut] * day[dish.name] for dish in dishes]) <= nutrient_limits[nut]*2

In [ ]:
#for cat in category_limits.keys():
#    for day in day_list:
#        prob += sum([day[dish.name]*dish.category[cat] for dish in dishes]) <= category_limits[cat][0]
#    prob += sum([sum([day[dish.name]*dish.category[cat] for dish in dishes]) for day in day_list]) <= category_limits[cat][1]
    

In [ ]:
new = branch_and_bound(prob)

In [ ]:
new.solve()

In [ ]:
print(pulp.LpStatus[new.status])

In [ ]:
print('Cost =', pulp.value(new.objective), '€')

In [ ]:
for day in ["_Mon_","_Tue_","_Wed_","_Thu_","_Fri_","_Sat_","_Sun_"]:
    print()
    print(day[1:4].upper())
    for v in new.variables():
        if v.varValue and day in v.name:
            print(v.name[9:], '=', '{0:.2f}'.format(v.varValue))
